# Network Introspector

This notebook had some code that was useful for me to debug a network.

There might be totally easier ways to do this, but eh, I learned a lot.

In [76]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../scripts/runners")

import numpy as np
import tensorflow as tf

import run_from_config as run
from layer_models import slow_get_result_of_layer_id

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
NETWORK_CONFIG_LOCATION = '../scripts/runners/config/capsnet_mtlsmallnorb_weight_decoder_loss.yaml'

tf.reset_default_graph()

args = run.parse_args(['--network-config', NETWORK_CONFIG_LOCATION, '--debug'])
network_config = run.parse_network_config(NETWORK_CONFIG_LOCATION, args)
lazy_batch_processor_pair = run.build_network(network_config)
variables = run.parameters_for_config_id(network_config)

lazy_batch_processor = lazy_batch_processor_pair.train
layer_features = lazy_batch_processor_pair.layer_features
input_placeholder_for_gpus = lazy_batch_processor_pair.input_placeholder_for_gpus
targets_placeholder_for_gpus = lazy_batch_processor_pair.targets_placeholder_for_gpus
is_training_placeholder = lazy_batch_processor_pair.is_training_placeholder
losses_ops = lazy_batch_processor_pair.losses_ops

Conv2DLayer(assert_output_shape=[40, 40, 256], id=None, input_id=None, filter_count=256, kernel_length=9, strides=1, padding=None, activation='leaky_relu') Tensor("build_losses_ops/losses_ops/features_0/capsnet_mtlsmallnorb_weight_decoder_loss/augment_data/Merge:0", shape=(50, 48, 48, 1), dtype=float32, device=/device:CPU:0)
PrimaryCapsule(assert_output_shape=[8192, 8], id=None, input_id=None, caps_count=32, caps_dim=8, kernel_length=9, strides=2, activation='leaky_relu') Tensor("build_losses_ops/losses_ops/features_0/capsnet_mtlsmallnorb_weight_decoder_loss/layer_0__Conv2DLayer/conv2d/LeakyRelu/Maximum:0", shape=(50, 40, 40, 256), dtype=float32, device=/device:CPU:0)
OutCapsule(assert_output_shape=[8192, 5, 16, 1], id=None, input_id=None, caps_count=5, caps_dim=16) Tensor("build_losses_ops/losses_ops/features_0/capsnet_mtlsmallnorb_weight_decoder_loss/layer_1__PrimaryCapsule/caps_1_squash/mul:0", shape=(50, 8192, 8), dtype=float32, device=/device:CPU:0)
DynamicRouting(assert_output_sh

## Print the variables

[n.name for n in tf.get_default_graph().as_graph_def().node]

## Force variables

This section assigns variables to some deterministic value (so don't use random initialization).

In [25]:
x = tf.placeholder(tf.float32, shape=(1024, 1024))

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]


### Force variables, and set an intermediate layer output value

In [78]:
# here's an example of grabbing the output from some layer
mask = slow_get_result_of_layer_id(layer_features, 'mask')
azimith_out = slow_get_result_of_layer_id(layer_features, 'azimith_output')

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    # lol make all the variables some specific value
    #variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    #for var in variables:
    #    var_val = np.arange(int(np.prod(var.shape))).reshape(var.shape) / int(np.prod(var.shape))        
    #    sess.run(var.assign(var_val))

    input_batch = np.ones(input_placeholder_for_gpus.shape)
    target_batch = np.ones(targets_placeholder_for_gpus.shape)
    out = sess.run(
        azimith_out,
        feed_dict={
            input_placeholder_for_gpus: input_batch,
            targets_placeholder_for_gpus: target_batch,
            is_training_placeholder: True,
            mask: np.ones((50, 1, 5, 16, 1)),
        })
    
    print(out)

[[ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]
 [ 0.27005124]]


In [79]:
from loss_functions import azimith_loss

azimith_targets_placeholder = tf.placeholder(tf.float32, shape=(1, 1))
azimith_preds_placeholder = tf.placeholder(tf.float32, shape=(1, 1))

azimith_loss_ops = azimith_loss(azimith_targets_placeholder, azimith_preds_placeholder, 'sin_squared')

In [80]:
with tf.Session() as sess:
    
    for i in range(18):
        pred = i * 20
        target = 60
        azimith_loss_val = sess.run(
             azimith_loss_ops, 
             {
                 azimith_preds_placeholder: [[pred]],
                 azimith_targets_placeholder: [[target]],
             }
         )
        print('pred ', pred, '\ttarget', target, '\tloss', azimith_loss_val)

pred  0 	target 60 	loss 0.25
pred  20 	target 60 	loss 0.116978
pred  40 	target 60 	loss 0.0301537
pred  60 	target 60 	loss 0.0
pred  80 	target 60 	loss 0.0301537
pred  100 	target 60 	loss 0.116978
pred  120 	target 60 	loss 0.25
pred  140 	target 60 	loss 0.413176
pred  160 	target 60 	loss 0.586824
pred  180 	target 60 	loss 0.75
pred  200 	target 60 	loss 0.883022
pred  220 	target 60 	loss 0.969846
pred  240 	target 60 	loss 1.0
pred  260 	target 60 	loss 0.969846
pred  280 	target 60 	loss 0.883022
pred  300 	target 60 	loss 0.75
pred  320 	target 60 	loss 0.586824
pred  340 	target 60 	loss 0.413176


In [81]:
with tf.Session() as sess:
    
    for i in range(18):
        pred = i * 20
        target = 60
        losses = sess.run(
            losses_ops, 
            {
                targets_placeholder_for_gpus: [[pred]],
                inputs_placeholder_for_gpus: [[target]],
            }
        )
        print('loss ', losses)


InvalidArgumentError: You must feed a value for placeholder tensor 'data-targets' with dtype int64 and shape [1,50,2]
	 [[Node: data-targets = Placeholder[dtype=DT_INT64, shape=[1,50,2], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'data-targets', defined at:
  File "/Users/jessica/miniconda3/envs/mlp/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/jessica/miniconda3/envs/mlp/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/jessica/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/jessica/miniconda3/envs/mlp/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/jessica/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/jes